# Jane Street Market Prediction Using XGBoost

# 1.Overview

## 1.1 Description
In a perfectly efficient market, buyers and sellers would have all the agency and information needed to make rational trading decisions. As a result, products would always remain at their “fair values” and never be undervalued or overpriced. However, financial markets are not perfectly efficient in the real world. Even if a strategy is profitable now, it may not be in the future, and market volatility makes it impossible to predict the profitability of any given trade with certainty. 

## 1.2 Evaluation
- Utility score <br>
    - Each row in the test set represents a trading opportunity for which **you will be predicting an `action` value, 1 to make the trade and 0 to pass on it.** Each trade `j` has an associated `weight` and `resp`, which represents a return.

$$p_i = \sum_j(weight_{ij} * resp_{ij} * action_{ij}),$$
$$ t = \frac{\sum p_i }{\sqrt{\sum p_i^2}} * \sqrt{\frac{250}{|i|}},$$
where |i| is the number of unique dates in the test set. The utility is 				then defined as:

$$u = min(max(t,0), 6)  \sum p_i.$$

>https://www.kaggle.com/renataghisloti/understanding-the-utility-score-function

- $p_i$

    - Each row or trading opportunity can be chosen (action == 1) or not (action == 0). The variable _pi_ is a indicator for each day _i_, showing how much return we got for that day. **Since we want to maximize u, we also want to maximize _pi_**. To do that, we have to select the **least amount of negative _resp_ values** as possible (since this is the only negative value in my equation and only value that would make the total sum of p going down) and maximize the positive number of positive _resp_ transactions we select.

- $t$

    - **_t_** is **larger** when the return for **each day is better distributed and has lower variation.** It is better to have returns uniformly divided among days than have all of your returns concentrated in just one day. It reminds me a little of a **_L1_** over **_L2_** situation, where the **_L2_** norm penalizes outliers more than **_L1_**. Basically, we want to select uniformly distributed distributed returns over days, maximizing our return but giving a penalty on choosing too many dates.

    - t is simply the annualized sharpe ratio assuming that there are 250 trading days in a year, an important risk adjusted performance measure in investing. If sharpe ratio is negative, utility is zero. A sharpe ratio higher than 6 is very unlikely, so it is capped at 6. The utility function overall try to maximize the product of sharpe ratio and total return.



>https://www.kaggle.com/vivekanandverma/eda-xgboost-hyperparameter-tuning

Market Basics: Financial market is a dynamic world where investors, speculators, traders, hedgers understand the market by different strategies and use the opportunities to make profit. They may use fundamental, technical analysis, sentimental analysis,etc. to place their bet. As data is growing, many professionals use data to understand and analyse previous trends and predict the future prices to book profit.

Competition Description: The dataset provided contains set of features, feature_{0...129},representing real stock market data. Each row in the dataset represents a trading opportunity, for which we will be predicting an action value: 1 to make the trade and 0 to pass on it. Each trade has an associated weight and resp, which together represents a return on the trade. In the training set, train.csv, you are provided a resp value, as well as several other resp_{1,2,3,4} values that represent returns over different time horizons.

In Test set we don't have resp value, and other resp_{1,2,3,4} data, so we have to use only feature_{0...129} to make prediction.

Trades with weight = 0 were intentionally included in the dataset for completeness, although such trades will not contribute towards the scoring evaluation. So we will ignore it.



# 2. Implementing

> https://www.kaggle.com/wongguoxuan/eda-pca-xgboost-classifier-for-beginners

> https://www.kaggle.com/eudmar/jane-street-eda-pca-ensemble-methods

> https://www.kaggle.com/muhammadmelsherbini/jane-street-extensive-eda-pca-starter


## 0) Reduce Memory Usage
> https://www.kaggle.com/sbunzini/reduce-memory-usage-by-75

In [2]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

In [ ]:
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

### Importing dataset

In [ ]:
train = reduce_memory_usage(train)

## 1) import library

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
import optuna
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 2) load and clean dataset

### Cleaning data

In [ ]:
# Drop rows with 'weight'=0 
# Trades with weight = 0 were intentionally included in the dataset for completeness, 
# although such trades will not contribute towards the scoring evaluation
train = train[train['weight']!=0]


# Create 'action' column (dependent variable)
# The 'action' column is defined as such because of the evaluation metric used for this project.
# We want to maximise the utility function and hence pi where pi=∑j(weightij∗respij∗actionij)
# Positive values of resp will increase pi
train['action'] = train['resp'].apply(lambda x:x>0).astype(int)

In [ ]:
train.describe()

In [ ]:
train.tail(100)

In [ ]:
features = [col for col in list(train.columns) if 'feature' in col]

In [ ]:
missing_values = pd.DataFrame()
missing_values['column'] = features
missing_values['num_missing'] = [train[i].isna().sum() for i in features]
missing_values.T

In [ ]:
TRADING_THRESHOLD = 0.500
#Checking Missing Values in the features
n_features = 45
nan_val = train.isna().sum()[train.isna().sum() > 0].sort_values(ascending=False)
print(nan_val)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))

sns.barplot(y = nan_val.index[0:n_features], 
            x = nan_val.values[0:n_features], 
            alpha = 0.8
           )

plt.title(f'NaN values of train dataset (Top {n_features})')
plt.xlabel('NaN values')
fig.savefig(f'nan_values_top_{n_features}_features.png')
plt.show()

### Creating Train and Test DataFrame

In [ ]:
X = train[features]
y = train['action']

# Next, we hold out part of the training data to form the hold-out validation set
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

In [ ]:
train_median = train_x.median()
# Impute medians in both training set and the hold-out validation set
train_x = train_x.fillna(train_median)
valid_x = valid_x.fillna(train_median)

In [ ]:
train_median

# 2. Exploratory data analysis

In [ ]:
# First, we want to check if the target class is balanced or unbalanced in the training data
sns.set_palette("colorblind")
ax = sns.barplot(train_y.value_counts().index, train_y.value_counts()/len(train_y))
ax.set_title("Proportion of trades with action=0 and action=1")
ax.set_ylabel("Percentage")
ax.set_xlabel("Action")
sns.despine();
# Target class is fairly balanced with almost 50% of trades corresponding to each action

In [ ]:
# Next, we plot a diagonal correlation heatmap to see if there are strong correlations between the features

# Compute the correlation matrix
corr = train_x.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(20, 230, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# There are strong correlations between several of the features

In [ ]:
p = features
p.append('resp')
len(p)

In [ ]:
x = train[p].corr()
x

In [ ]:
x = x.abs()
upper = x.where(np.triu(np.ones(x.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)

In [ ]:
train.drop(to_drop, 1, inplace=True)
train

In [ ]:
#Resp Analysis
#Last subplot doesn't mean anything
resp_df = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
fig, axes = plt.subplots(nrows=2
                         , ncols=3,figsize=(20,10))
for i, column in enumerate(resp_df):
    sns.distplot(train[column],ax=axes[i//3,i%3])

In [ ]:
# Cumulative return analysis
fig, ax = plt.subplots(figsize=(16, 8))

resp = train['resp'].cumsum()
resp_1 = train['resp_1'].cumsum()
resp_2 = train['resp_2'].cumsum()
resp_3 = train['resp_3'].cumsum()
resp_4 = train['resp_4'].cumsum()

resp.plot(linewidth=2)
resp_1.plot(linewidth=2)
resp_2.plot(linewidth=2)
resp_3.plot(linewidth=2)
resp_4.plot(linewidth=2)

ax.set_xlabel ("Trade", fontsize=12)
ax.set_title ("Cumulative Trade Returns", fontsize=18)

plt.legend(loc="upper left");

resp and resp_4 variable are closely related so we can use this to set our 'action' variable.

# 3. Principal Components Analysis

In [ ]:
"""# Before we perform PCA, we need to normalise the features so that they have zero mean and unit variance
scaler = StandardScaler()
scaler.fit(train_x)
train_x_norm = scaler.transform(train_x)

pca = PCA()
comp = pca.fit(train_x_norm)"""

In [ ]:
"""# We plot a graph to show how the explained variation in the 129 features varies with the number of principal components
plt.plot(np.cumsum(comp.explained_variance_ratio_))
plt.grid()
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance')
sns.despine();

# The first 15 principal components explains about 80% of the variation
# The first 40 principal components explains about 95% of the variation"""

In [ ]:
"""# Using the first 50 principal components, we apply the PCA mapping
# From here on, we work with only 50 features instead of the full set of 129 features
pca = PCA(n_components=50).fit(train_x_norm)
train_x_transform = pca.transform(train_x_norm)"""

In [ ]:
"""# Transform the validation set
valid_x_transform = pca.transform(scaler.transform(valid_x))"""

# 4. Train XGBoost classifier + Tune hyperparameters using Optuna

In [ ]:
# We create the XGboost-specific DMatrix data format from the numpy array. 
# This data structure is optimised for memory efficiency and training speed
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(valid_x, label=valid_y)

In [ ]:
# The objective function is passed an Optuna specific argument of trial
def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 25),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
# trials will be evaluated based on their accuracy on the test set
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
best_params = trial.params
best_params['tree_method'] = 'gpu_hist' 
best_params['objective'] = 'binary:logistic'

In [ ]:
# Fit the XGBoost classifier with optimal hyperparameters
optimal_clf = xgb.XGBClassifier(**best_params)

In [ ]:
optimal_clf.fit(train_x, train_y)

In [ ]:
# Plot how the best accuracy evolves with number of trials
fig = optuna.visualization.plot_optimization_history(study)
fig.show();

In [ ]:
# We can also plot the relative importance of different hyperparameter settings
fig = optuna.visualization.plot_param_importances(study)
fig.show();

# 5.Fit classifier on test set

In [ ]:
# We impute the missing values with the medians
def fillna_npwhere(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
from tqdm import tqdm
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
"""for (test_df, sample_prediction_df) in iter_test:
    wt = test_df.iloc[0].weight
    if(wt == 0):
        sample_prediction_df.action = 0 
    else:
        sample_prediction_df.action = optimal_clf.predict(pca.transform(scaler.transform(fillna_npwhere(test_df[features].values,train_median[features].values))))
    env.predict(sample_prediction_df)"""

In [ ]:
for (test_df, pred_df) in iter_test:
    if test_df['weight'].item() > 0:
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        y_preds = clf.predict(X_test)
        pred_df.action = y_preds
    else:
        pred_df.action = 0
    env.predict(pred_df)

reference

>    https://www.kaggle.com/wongguoxuan/eda-pca-xgboost-classifier-for-beginners
>    https://www.kaggle.com/vivekanandverma/eda-xgboost-hyperparameter-tuning